# CFB Ranking
## The Code

Import requirements

In [1]:
import pandas as pd

Here, we initialize values and methods used to manipulate CFB Model<br>
For now that's just the dataframe ofc, there will be more

In [2]:
class CFBModel:
    def __init__(self):
        # initialize games dataframe with headers
        self.df_list = []
        self.games_df = pd.DataFrame()

    def csv_to_df(self, file):
        # set first row of csv as dataframe headers
        df = pd.read_csv(file, encoding='ANSI')
        df.rename(columns=df.iloc[0])
        # concatenate all years of data
        self.df_list.append(df)
        self.games_df = pd.concat(self.df_list)

Build the CFB model with the scraped game data

In [3]:
cfb_model = CFBModel()
for year in range(2010, 2021):
    cfb_model.csv_to_df("../data/games/{}_games.csv".format(year))